In [1]:
nbit = 4
# llamasize = '7B'
llamasize = '13B'
temp = 0.1

In [2]:
import pandas as pd
import glob
import pickle

In [3]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx"

output_data_path = base_path + "/../Llama-2/data/output/2023.11.27-2_LLama2_13B-chat-hf_4-bit_float-16_3-few-shot_SEMANTIC_SIMILARITY"

drive.mount(mount_point, force_remount=True)
# data_path = '/content/gdrive/MyDrive/Colab/SDG/Data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx'

Mounted at /content/gdrive


In [4]:
data = pd.read_excel(input_data_path)
data.head(3)

,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN


In [5]:
data_sel = data

In [6]:
files = glob.glob(output_data_path + "/*.pkl")
# files

In [7]:
results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

In [8]:
len(results)

1021

In [9]:
results_processed = pd.DataFrame(results, columns=['Id', 'text_id', 'sdg', 'result'])
results_processed.head(3)

,Id,text_id,sdg,result
0,22,6576,0,---SDG 3---
1,23,3912,0,"\nInput text: ""---SDG 0---"""
2,24,1565,0,---SDG 0---


In [10]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed.head(3)

,Id,text_id,sdg,result,label
0,22,6576,0,---SDG 3---,3
1,23,3912,0,"\nInput text: ""---SDG 0---""",0
2,24,1565,0,---SDG 0---,0


In [11]:
final = data_sel.merge(results_processed.drop(columns=['sdg']), left_on='text_id', right_on='text_id')
final.head(3)

,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,Id,result,label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,---SDG 12---,12
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,---SDG 16---,16


In [12]:
final.shape[0]

1021

In [13]:
# final.groupby(['label']).count()

In [14]:
# final.label.values

In [15]:
final = final[~final.label.isna()]

In [16]:
# final.to_excel(output_data_path+"/2023.11.27-2_LLama2_13B-chat-hf_4-bit_float-16_3-few-shot_SEMANTIC_SIMILARITY.xlsx")

In [17]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1019
1019
              precision    recall  f1-score   support

           0       1.00      0.42      0.60        59
           1       0.76      0.90      0.82        60
           2       0.95      0.87      0.90        60
           3       0.80      0.92      0.85        60
           4       0.86      0.95      0.90        60
           5       0.89      0.90      0.89        60
           6       1.00      0.92      0.96        60
           7       0.92      0.80      0.86        60
           8       0.58      0.70      0.64        60
           9       0.82      0.77      0.79        60
          10       0.84      0.63      0.72        60
          11       0.80      0.87      0.83        60
          12       0.78      0.95      0.86        60
          13       0.89      0.90      0.89        60
          14       0.97      0.95      0.96        60
          15       0.89      0.95      0.92        60
          16       0.87      0.92      0.89        60
          17     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# classification_report_filename = output_data_path+f'/sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}_classification_report.xlsx'
# classification_report_filename

In [19]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [20]:
# test = pd.read_excel(classification_report_filename)
# test

In [21]:
final

,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,Id,result,label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,---SDG 12---,12
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,---SDG 16---,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,---SDG 0---,0
4,4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,---SDG 7---,7
...,...,...,...,...,...,...,...,...,...,...,...
1016,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1016,---SDG 16---,16
1017,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1017,---SDG 16---,16
1018,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1018,---SDG 16---,16
1019,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1019,---SDG 16---,16


In [22]:
final_new = data_sel.merge(results_processed.drop(columns=['sdg']), left_on='text_id', right_on='text_id')
final_new.head()

,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,Id,result,label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,---SDG 11---,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,---SDG 12---,12
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,---SDG 16---,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,---SDG 0---,0
4,4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,---SDG 7---,7


In [23]:
final_new.shape[0]

1021

In [24]:
# final_new = final

In [25]:
results_processed_merged = final_new
results_processed_merged = results_processed_merged.rename(columns={'label': 'predicted_label_clean'})

In [26]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
        # new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0 #new_value

In [27]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.79      0.45      0.57        60
           1       0.76      0.90      0.82        60
           2       0.95      0.87      0.90        60
           3       0.80      0.92      0.85        60
           4       0.86      0.95      0.90        60
           5       0.89      0.90      0.89        60
           6       1.00      0.92      0.96        60
           7       0.92      0.80      0.86        60
           8       0.58      0.70      0.64        60
           9       0.82      0.77      0.79        60
          10       0.84      0.62      0.71        60
          11       0.80      0.87      0.83        60
          12       0.78      0.95      0.86        60
          13       0.89      0.90      0.89        60
          14       0.97      0.95      0.96        60
          15       0.89      0.95      0.92        60
          16       0.87      0.92      0.89        60

    accuracy    

In [28]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100

In [29]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [30]:
print(precision, recall, accuracy, f1_score)

84.76869172631024 84.21568627450979 84.2156862745098 83.89001421634428


In [31]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

84.8 84.2 84.2 83.9
